The json file contains metadata from the website https://facethefuture.com/#projects ,and the scripts shows preprocessing and shapefile generation

In [11]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
df_ftf_2 = pd.read_json("Forest_Monitoring/input/FaceTheFuture/FaceTheFuture.json")
df_ftf_2.tail()

In [ ]:
# Expanding the  'properties' column to extract information in it in separate columns
properties_df = df_ftf_2["properties"].apply(pd.Series)


df_expanded = pd.concat([df_ftf_2.drop(columns=["properties"]), properties_df], axis=1)


df_expanded.tail()

In [ ]:


# Filtering the specified columns with important information for our analysis
filtered_columns = ["GlobalID", "id", "TreeNumber", "ContractArea", "PlantingYear_no", "Comments", "Creator", "geometry"]
df_filtered = df_expanded[filtered_columns]


df_filtered.tail()

In [ ]:
# Renaming the columns
new_column_names = {
    "GlobalID": "project_id_reported",
    "id": "site_id_reported",
    "TreeNumber": "trees_planted_reported",
    "ContractArea": "site_sqkm",
    "PlantingYear_no": "planting_date_reported",
    "Comments": "description",
    "Creator": "host_name",
    "geometry": "geometry_reported"
}

df_filtered = df_filtered.rename(columns=new_column_names)

df_filtered.tail()

In [ ]:
import geopandas as gpd

from shapely.geometry import shape



def safe_shape(geom):
    try:
        return shape(geom)
    except Exception as e:
        print(f"Error processing geometry: {geom}, Error: {e}")
        return None

# Filtering out None values in 'geometry_reported'
df_filtered = df_filtered[df_filtered['geometry_reported'].notnull()]


df_filtered['geometry'] = df_filtered['geometry_reported'].apply(safe_shape)


df_filtered = df_filtered[df_filtered['geometry'].notnull()]


gdf = gpd.GeoDataFrame(df_filtered, geometry='geometry')

# Exploding the MultiPolygon into individual Polygons
gdf_exploded = gdf.explode(index_parts=False)

# Saving the GeoDataFrame as a shapefile
gdf_exploded.to_file("Forest_Monitoring/input/FaceTheFuture/FaceTheFuture.shp")


df_exploded = pd.DataFrame(gdf_exploded)
df_exploded.head()